<a href="https://colab.research.google.com/github/WaSilveira/Sis_Reco_ImagemD.ipynb/blob/main/Sis_Reco_ImagemD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
```python
# Configuração do Ambiente no Google Colab
# Certifique-se de que você tem GPU disponível. Vá para Runtime > Change runtime type e selecione GPU.

# Importação de Bibliotecas Necessárias
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
from shutil import move
from tqdm import tqdm
from sklearn.neighbors import NearestNeighbors
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Montando o Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Configuração da API do Kaggle
# Substitua "<your kaggle username>" e "<your kaggle api key>" pelos seus próprios dados
kaggle_json = """{"username": "<your kaggle username>", "key":"<your kaggle api key>"}"""
with open('kaggle.json', 'w') as file:
    file.write(kaggle_json)

!pip install --upgrade --force-reinstall --no-deps kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Baixando e descompactando o dataset do Kaggle
!kaggle datasets download -d paramaggarwal/fashion-product-images-small
!unzip fashion-product-images-small.zip

# Preparação dos Dados
os.mkdir('/content/Fashion_data')
os.chdir('/content/Fashion_data')
df = pd.read_csv('/content/styles.csv', usecols=['id', 'masterCategory']).reset_index()
df['id'] = df['id'].astype('str')
all_images = os.listdir('/content/images/')
os.mkdir('/content/Fashion_data/categories')

for image in tqdm(all_images):
    category = df[df['id'] == image.split('.')[0]]['masterCategory']
    category = str(list(category)[0])
    if not os.path.exists(os.path.join('/content/Fashion_data/categories', category)):
        os.mkdir(os.path.join('/content/Fashion_data/categories', category))
    path_from = os.path.join('/content/images', image)
    path_to = os.path.join('/content/Fashion_data/categories', category, image)
    move(path_from, path_to)

# Definindo o caminho para os dados
data_dir = '/content/Fashion_data/categories'
train_dir = os.path.join(data_dir, 'train')
validation_dir = os.path.join(data_dir, 'validation')

IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32

train_dataset = image_dataset_from_directory(train_dir, image_size=IMAGE_SIZE, batch_size=BATCH_SIZE, label_mode=None)
validation_dataset = image_dataset_from_directory(validation_dir, image_size=IMAGE_SIZE, batch_size=BATCH_SIZE, label_mode=None)

# Definição do Modelo
MODULE_HANDLE = "https://tfhub.dev/google/bit/m-r50x3/1"
feature_extractor = hub.KerasLayer(MODULE_HANDLE, input_shape=IMAGE_SIZE + (3,), trainable=False)

# Extração de Recursos das Imagens
def extract_features(dataset):
    features = []
    for batch in dataset:
        batch_features = feature_extractor(batch)
        features.append(batch_features)
    return np.vstack(features)

train_features = extract_features(train_dataset)
validation_features = extract_features(validation_dataset)

# Implementação do Sistema de Recomendação
num_neighbors = 5
neighbors = NearestNeighbors(n_neighbors=num_neighbors, algorithm='auto').fit(train_features)

def recommend(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=IMAGE_SIZE)
    img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    image_features = feature_extractor(img_array)
    distances, indices = neighbors.kneighbors(image_features)

    recommended_images = []
    for idx in indices[0]:
        image_path = train_dataset.file_paths[idx]
        recommended_images.append(image_path)

    return recommended_images

# Testando o Sistema de Recomendação
# Exemplo de uso
image_path = "/content/drive/My Drive/caminho/para/sua/imagem.jpg"
recommendations = recommend(image_path)

print("Imagens recomendadas:")
for img_path in recommendations:
    print(img_path)
```